# ⚙️ SmartBin — Optimització CNN amb Optuna i dataset reorganitzat *(Versió 2.2)*

> **Aquest notebook ha estat generat amb assistència d’intel·ligència artificial, però totes les decisions sobre les tècniques i metodologies utilitzades han estat preses per una persona humana.**

---

## 🧩 Descripció general

Aquesta versió del projecte **SmartBin** representa un pas endavant en la millora del sistema de classificació automàtica de residus, centrant-se en l’**optimització d’una xarxa neuronal convolucional (CNN)** mitjançant la llibreria **Optuna**.  
El model ha estat adaptat a un **nou conjunt de dades reorganitzat** en format pla, amb quatre categories principals de residus:

- **Envasos**  
- **Paper**  
- **Rebuig**  
- **Vidre**

Aquesta estructura simplificada permet una gestió més eficient de les dades i un entrenament més coherent del model.

> **Dataset utilitzat:** [Recyclable and Household Waste Classification (Kaggle)](https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification)

---

## 🎯 Objectiu

L’objectiu principal d’aquesta versió és **millorar la precisió i l’equilibri del model** de classificació d’imatges per a la Paperera Intel·ligent (*SmartBin*), aplicant tècniques avançades com:

- **Augmentació dinàmica de dades** per incrementar la varietat d’imatges durant l’entrenament.  
- **Pesatge de classes (class weighting)** per compensar el desequilibri del dataset.  
- **Cerca automàtica d’hiperparàmetres amb Optuna**, garantint una optimització més eficient i menys dependent d’assaig i error manual.

Aquest enfocament busca aconseguir un model robust i adaptable, capaç de mantenir un rendiment estable fins i tot amb variacions en les condicions de captura d’imatges del SmartBin real.

---

## 🗂️ Dataset reorganitzat

Les imatges s’han estructurat en un **format pla** per simplificar el flux d’entrenament i evitar jerarquies innecessàries.  
L’organització és la següent:

images_reorganitzades/
├── envasos/
├── paper/
├── rebuig/
└── vidre/

yaml
Copia el codi

La divisió automàtica de dades s’ha realitzat amb la proporció estàndard:
- **Entrenament:** 60%  
- **Validació:** 20%  
- **Test:** 20%

Per contrarestar possibles desequilibris entre classes (p. ex. més imatges d’envasos que de vidre), s’utilitza un **WeightedRandomSampler**, que assegura una representació equilibrada durant l’entrenament.

---

## 🧠 Arquitectura del model

El model utilitzat és una **xarxa neuronal convolucional lleugera i eficient**, dissenyada per funcionar en entorns físics amb recursos limitats, com el SmartBin.

| Capa | Tipus | Descripció |
|------|-------|------------|
| **Conv1** | Convolucional | 3→32 filtres, *kernel* 3×3, activació ReLU i MaxPooling |
| **Conv2** | Convolucional | 32→64 filtres, *kernel* 3×3, ReLU i MaxPooling |
| **FC1** | Fully Connected | 512 neurones, activació ReLU i Dropout |
| **FC2** | Sortida | 4 neurones (una per classe) |

Aquesta arquitectura combina senzillesa i potència, permetent un bon compromís entre precisió i temps de càlcul.

---

## ⚙️ Optimització amb Optuna

Per trobar la millor configuració d’hiperparàmetres, s’ha utilitzat **Optuna**, una llibreria d’optimització bayesiana basada en *Tree-structured Parzen Estimators (TPE)*, combinada amb el mètode **Median Pruner** per eliminar proves poc prometedores.

Durant el procés, s’han executat **20 proves (trials)** amb l’objectiu de **minimitzar la pèrdua de validació** (*validation loss*).

| Hiperparàmetre | Rang de cerca |
|-----------------|---------------|
| `learning_rate` | 0.0001 – 0.002 |
| `batch_size` | 16, 24, 32, 48 |
| `num_epochs` | 4 – 10 |
| `dropout` | 0.3 – 0.7 |
| `weight_factor` | 0.5 – 2.0 |

### 🔬 Altres tècniques aplicades
- **Early Stopping:** amb una paciència de 4 èpoques per evitar sobreentrenament.  
- **Augmentació dinàmica d’imatges:** rotacions, retalls i canvis d’il·luminació durant l’entrenament.  
- **Scheduler automàtic del learning rate:** ajusta progressivament la taxa d’aprenentatge per millorar la convergència.

Aquest conjunt de tècniques garanteix una optimització més estable i adaptada a la naturalesa variable del dataset.








In [1]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms
from PIL import Image
import os
import random
import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm
import json
from datetime import datetime
from collections import Counter

# Dataset Class Millorada per Dataset Pla
class WasteDatasetFlat(Dataset):
    def __init__(self, root_dir, split, transform=None, val_transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.val_transform = val_transform  # Transformacions sense augmentació per val/test
        self.split = split
        
        # Classes finals reorganitzades
        self.classes = ['envasos', 'paper', 'rebuig', 'vidre']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        self.image_paths = []
        self.labels = []
        
        print(f"   🔄 Creant dataset {split} amb estructura plana...")
        
        # Carregar imatges de la estructura plana reorganitzada
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.exists(class_dir):
                print(f"   ⚠️ Advertència: No es troba la carpeta {class_dir}")
                continue
                
            # Obtenir totes les imatges de la classe
            image_names = []
            for file in os.listdir(class_dir):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_names.append(file)
            
            # Barrejar per obtenir splits consistents
            random.seed(42)  # Seed fix per reproducibilitat
            random.shuffle(image_names)
            
            # Dividir segons split (60/20/20)
            total_images = len(image_names)
            train_end = int(0.6 * total_images)
            val_end = int(0.8 * total_images)
            
            if split == 'train':
                selected_images = image_names[:train_end]
            elif split == 'val':
                selected_images = image_names[train_end:val_end]
            else:  # test
                selected_images = image_names[val_end:]
            
            # Afegir al dataset
            for image_name in selected_images:
                full_path = os.path.join(class_dir, image_name)
                self.image_paths.append(full_path)
                self.labels.append(self.class_to_idx[class_name])
        
        # Estadístiques del dataset
        label_counts = Counter(self.labels)
        class_distribution = {self.classes[i]: label_counts[i] for i in range(len(self.classes))}
        
        print(f"   ✅ Dataset {split} creat: {len(self.image_paths)} imatges")
        print(f"   📊 Distribució per classe: {class_distribution}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error carregant {image_path}: {e}")
            # Retornar una imatge negra com a fallback
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        # Aplicar transformacions segons el split
        if self.transform and self.split == 'train':
            image = self.transform(image)
        elif self.val_transform and self.split in ['val', 'test']:
            image = self.val_transform(image)
        
        return image, label
    
    def get_class_weights(self):
        """Calcula els pesos per WeightedRandomSampler"""
        label_counts = Counter(self.labels)
        total_samples = len(self.labels)
        
        # Calcular pesos inversament proporcionals a la freqüència
        class_weights = []
        for i in range(len(self.classes)):
            weight = total_samples / (len(self.classes) * label_counts[i])
            class_weights.append(weight)
        
        # Crear pesos per cada mostra
        sample_weights = [class_weights[label] for label in self.labels]
        
        return torch.DoubleTensor(sample_weights)

# CNN Model (mantenint el mateix que tenies)
class CNN(nn.Module):
    def __init__(self, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def evaluate_model(model, test_loader, criterion, device, class_names):
    """Evalúa el modelo y retorna métricas detalladas"""
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0.0
    
    print("🔍 Evaluant model en dataset de test...")
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Avaluació"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    
    return {
        'test_loss': test_loss,
        'accuracy': accuracy,
        'predictions': all_preds,
        'true_labels': all_labels,
        'classification_report': classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    }

def plot_confusion_matrix(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    """Genera i guarda la matriu de confusió"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriu de Confusió - Millor Model', fontsize=16)
    plt.ylabel('Etiquetes Reals', fontsize=12)
    plt.xlabel('Prediccions', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 Matriu de confusió guardada a: {save_path}")

def generate_report(study, best_metrics, class_names, save_path='optimization_report.json'):
    """Genera un informe complet de l'optimització"""
    report = {
        'optimization_info': {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'project': 'Paperera Intel·ligent - Classificació de Residus',
            'dataset_info': {
                'classes': class_names,
                'total_classes': len(class_names),
                'data_split': '60/20/20 (train/val/test)'
            },
            'n_trials': len(study.trials),
            'best_trial_number': study.best_trial.number,
            'optimization_direction': 'minimize_validation_loss'
        },
        'best_hyperparameters': study.best_trial.params,
        'best_validation_loss': study.best_trial.value,
        'test_metrics': {
            'test_loss': best_metrics['test_loss'],
            'test_accuracy': best_metrics['accuracy']
        },
        'classification_report': best_metrics['classification_report'],
        'trial_history': [
            {
                'trial': t.number,
                'params': t.params,
                'value': t.value,
                'state': str(t.state)
            } for t in study.trials if t.value is not None
        ]
    }
    
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print(f"📄 Informe complet guardat a: {save_path}")
    return report

def plot_optimization_history(study, save_path='optimization_history.png'):
    """Genera gràfic de la història d'optimització"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Història de valors
    trial_numbers = [t.number for t in study.trials if t.value is not None]
    values = [t.value for t in study.trials if t.value is not None]
    
    ax1.plot(trial_numbers, values, 'b-o', markersize=4)
    ax1.axhline(y=study.best_value, color='r', linestyle='--', label=f'Millor: {study.best_value:.4f}')
    ax1.set_xlabel('Trial')
    ax1.set_ylabel('Validation Loss')
    ax1.set_title('Història d\'Optimització')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Distribució de valors
    if len(values) > 0:
        ax2.hist(values, bins=min(20, len(values)), alpha=0.7, color='skyblue', edgecolor='black')
        ax2.axvline(x=study.best_value, color='r', linestyle='--', label=f'Millor: {study.best_value:.4f}')
    ax2.set_xlabel('Validation Loss')
    ax2.set_ylabel('Freqüència')
    ax2.set_title('Distribució de Resultats')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📈 Gràfic d'optimització guardat a: {save_path}")

def plot_class_distribution(train_dataset, save_path='class_distribution.png'):
    """Genera gràfic de distribució de classes"""
    label_counts = Counter(train_dataset.labels)
    class_names = train_dataset.classes
    counts = [label_counts[i] for i in range(len(class_names))]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(class_names, counts, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    
    # Afegir valors sobre les barres
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
                str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.title('Distribució de Classes en Dataset d\'Entrenament', fontsize=14)
    plt.xlabel('Classes de Residus')
    plt.ylabel('Nombre d\'Imatges')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 Gràfic de distribució guardat a: {save_path}")

# Configuració
print("🚀 INICIANDO OPTUNA HYPERPARAMETER TUNING - PAPERERA INTEL·LIGENT")
print("="*70)

# Transformacions amb augmentació dinàmica per entrenament
train_transform = transforms.Compose([
    transforms.Resize((240, 240)),  # Més gran per després fer crop
    transforms.RandomCrop((224, 224)),  # Crop aleatori
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Transformacions sense augmentació per validació i test
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Inicialitzar datasets
print("📂 Carregant datasets amb estructura reorganitzada...")
dataset_path = 'images_reorganitzades'  # Nova ruta amb estructura plana

train_dataset = WasteDatasetFlat(dataset_path, 'train', 
                                transform=train_transform, 
                                val_transform=val_test_transform)
val_dataset = WasteDatasetFlat(dataset_path, 'val', 
                              val_transform=val_test_transform)
test_dataset = WasteDatasetFlat(dataset_path, 'test', 
                               val_transform=val_test_transform)

print(f"📊 Dataset: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)} | {len(train_dataset.classes)} classes")
print(f"🏷️ Classes: {train_dataset.classes}")

# Generar gràfic de distribució
plot_class_distribution(train_dataset)

# Calcular pesos per WeightedRandomSampler
print("⚖️ Calculant pesos per equilibrar classes...")
sample_weights = train_dataset.get_class_weights()
print(f"   📈 Pesos calculats per {len(sample_weights)} mostres d'entrenament")

# Variables globals per tracking
best_global_loss = float('inf')
best_model_state = None
trial_count = 0

def objective(trial):
    global best_global_loss, best_model_state, trial_count
    trial_count += 1
    
    # Hiperparàmetres optimitzats per dataset desequilibrat
    lr = trial.suggest_float('learning_rate', 0.0001, 0.002, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 24, 32, 48])
    epochs = trial.suggest_int('num_epochs', 4, 10)
    dropout = trial.suggest_float('dropout', 0.3, 0.7, step=0.1)
    
    # Nou paràmetre: factor de pes per classes
    weight_factor = trial.suggest_float('weight_factor', 0.5, 2.0, step=0.1)
    
    print(f"\n🔍 Trial {trial_count}/20")
    print(f"   📊 Paràmetres: lr={lr:.5f}, bs={batch_size}, epochs={epochs}, dropout={dropout:.1f}, weight_factor={weight_factor:.1f}")
    print(f"   🎯 Baseline a superar: validation loss anterior")
    
    # Crear DataLoaders amb WeightedRandomSampler per train
    # Aplicar factor de pes als sample weights
    adjusted_weights = sample_weights * weight_factor
    
    weighted_sampler = WeightedRandomSampler(
        weights=adjusted_weights,
        num_samples=len(adjusted_weights),
        replacement=True  # Permetre repeticions per oversampling
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        sampler=weighted_sampler,  # Usar sampler en lloc de shuffle
        num_workers=0, 
        pin_memory=False
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=0, 
        pin_memory=False
    )
    
    # Model setup amb dropout personalitzable
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNN(len(train_dataset.classes), dropout=dropout).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training loop
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 4  # Augmentem paciència per models més complexos
    
    print(f"   🚀 Iniciant entrenament amb WeightedSampler...")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_bar = tqdm(train_loader, desc=f"   Epoch {epoch+1}/{epochs} [Train]", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Actualitzar barra de progrés
            current_train_loss = train_loss / (train_bar.n + 1)
            current_train_acc = 100. * train_correct / train_total
            train_bar.set_postfix({'Loss': f'{current_train_loss:.4f}', 'Acc': f'{current_train_acc:.1f}%'})
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100. * train_correct / train_total
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"   Epoch {epoch+1}/{epochs} [Val]  ", leave=False)
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Actualitzar barra de progrés
                current_val_loss = val_loss / (val_bar.n + 1)
                current_val_acc = 100. * val_correct / val_total
                val_bar.set_postfix({'Loss': f'{current_val_loss:.4f}', 'Acc': f'{current_val_acc:.1f}%'})
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100. * val_correct / val_total
        
        # Mostrar resultats de l'epoch
        improvement_indicator = ""
        if avg_val_loss < best_val_loss:
            improvement_indicator = " ⬆️"
        
        print(f"   Epoch {epoch+1}: Train={avg_train_loss:.4f}({train_accuracy:.1f}%) | Val={avg_val_loss:.4f}({val_accuracy:.1f}%){improvement_indicator}")
        
        # Report intermediate result per pruning
        trial.report(avg_val_loss, epoch)
        
        # Prune si no és prometedor
        if trial.should_prune():
            print(f"   ✂️ Trial podat - no és prometedor")
            raise optuna.TrialPruned()
        
        # Update best validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            
            # Si és el millor model global, guardar-lo
            if avg_val_loss < best_global_loss:
                best_global_loss = avg_val_loss
                best_model_state = model.state_dict().copy()
                torch.save(best_model_state, 'best_model_temp.pth')
                print(f"   🏆 ¡NOU RÈCORD GLOBAL! Val Loss: {avg_val_loss:.6f}")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= patience:
            print(f"   ⏰ Early stopping (sense millora per {patience} epochs)")
            break
    
    print(f"   ✅ COMPLETAT - Millor Val Loss: {best_val_loss:.6f}")
    
    # Cleanup
    del model, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()
    
    return best_val_loss

# Crear estudi d'Optuna
print(f"🎯 Objectiu: Optimitzar classificació de residus amb dataset equilibrat")
print(f"💻 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print(f"⚖️ Estratègia: WeightedRandomSampler per equilibrar classes")
print(f"🔄 Augmentació: Dinàmica només en entrenament")
print(f"⏱️ Temps estimat: 3-4 hores per 20 trials")

study = optuna.create_study(
    direction='minimize',
    study_name='waste_classification_optimization',
    sampler=optuna.samplers.TPESampler(n_startup_trials=5),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=2)
)

# Executar optimització
print("🚀 Iniciant optimització...")
try:
    study.optimize(objective, n_trials=20, timeout=14400)  # 20 trials, 4h max
    optimization_completed = True
except KeyboardInterrupt:
    print("⏸️ Interromput per usuari")
    optimization_completed = True
except Exception as e:
    print(f"❌ Error durant optimització: {e}")
    optimization_completed = False

# Resultats finals
print("\n" + "="*70)
print("🎉 OPTIMITZACIÓ COMPLETADA - PAPERERA INTEL·LIGENT")
print("="*70)

if len(study.trials) > 0 and study.best_trial:
    print(f"🏆 MILLOR RESULTAT:")
    print(f"   📊 Validation Loss: {study.best_trial.value:.6f}")
    print(f"   ⚙️ Paràmetres: {study.best_trial.params}")
    print(f"   🔢 Trial número: {study.best_trial.number}")
    
    print(f"\n📈 Resum de trials:")
    print(f"   ✅ Trials completats: {len([t for t in study.trials if t.value is not None])}")
    print(f"   ✂️ Trials podats: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")
    print(f"   ❌ Trials fallits: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")
    
    # Evaluar el millor model en test set
    if os.path.exists('best_model_temp.pth'):
        print(f"\n🧪 AVALUACIÓ EN TEST SET:")
        print("-" * 50)
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        best_model = CNN(len(train_dataset.classes), dropout=study.best_trial.params.get('dropout', 0.5)).to(device)
        best_model.load_state_dict(torch.load('best_model_temp.pth'))
        
        # Crear test loader sense weighted sampler
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
        criterion = nn.CrossEntropyLoss()
        
        test_metrics = evaluate_model(best_model, test_loader, criterion, device, train_dataset.classes)
        
        print(f"📊 Test Loss: {test_metrics['test_loss']:.6f}")
        print(f"🎯 Test Accuracy: {test_metrics['accuracy']*100:.2f}%")
        
        # Mostrar accuracy per classe
        print(f"\n📋 ACCURACY PER CLASSE:")
        class_report = test_metrics['classification_report']
        for class_name in train_dataset.classes:
            if class_name in class_report:
                precision = class_report[class_name]['precision']
                recall = class_report[class_name]['recall']
                f1 = class_report[class_name]['f1-score']
                print(f"   {class_name:8}: P={precision:.3f} R={recall:.3f} F1={f1:.3f}")
        
        # Generar tots els informes i gràfics
        print(f"\n💾 GENERANT INFORMES:")
        print("-" * 50)
        
        plot_confusion_matrix(test_metrics['true_labels'], test_metrics['predictions'], 
                            train_dataset.classes, 'paperera_confusion_matrix.png')
        
        generate_report(study, test_metrics, train_dataset.classes, 'paperera_optimization_report.json')
        
        plot_optimization_history(study, 'paperera_optimization_history.png')
        
        # Guardar model final amb nom descriptiu
        model_filename = f"paperera_model_final_acc{test_metrics['accuracy']*100:.1f}.pth"
        torch.save(best_model.state_dict(), model_filename)
        
        # Guardar també configuració del model
        model_config = {
            'num_classes': len(train_dataset.classes),
            'classes': train_dataset.classes,
            'dropout': study.best_trial.params.get('dropout', 0.5),
            'best_hyperparameters': study.best_trial.params,
            'test_accuracy': test_metrics['accuracy'],
            'test_loss': test_metrics['test_loss'],
            'model_architecture': 'CNN_2Conv_2FC'
        }
        
        with open('paperera_model_config.json', 'w', encoding='utf-8') as f:
            json.dump(model_config, f, indent=2, ensure_ascii=False)
        
        print("✅ Arxius generats:")
        print(f"   📦 {model_filename} - Millor model optimitzat")
        print("   🔧 paperera_model_config.json - Configuració del model")
        print("   📊 paperera_confusion_matrix.png - Matriu de confusió")
        print("   📄 paperera_optimization_report.json - Informe complet")
        print("   📈 paperera_optimization_history.png - Història d'optimització")
        print("   📊 class_distribution.png - Distribució de classes")
        
        # Netejar arxiu temporal
        if os.path.exists('best_model_temp.pth'):
            os.remove('best_model_temp.pth')
        
        print(f"\n🎊 ¡PROCÉS COMPLETAT EXITOSAMENT!")
        
        # Mostrar recomanacions per la paperera
        print(f"\n💡 RECOMANACIONS PER LA PAPERERA INTEL·LIGENT:")
        print("-" * 50)
        print("✅ Model optimitzat amb WeightedSampler per equilibrar classes")
        print("✅ Augmentació dinàmica aplicada per millorar generalització")
        print("✅ Dataset reorganitzat en 4 categories principals de residus")
        
        # Anàlisi de rendiment per categoria
        worst_class = min(class_report.keys(), 
                         key=lambda x: class_report[x]['f1-score'] if x in train_dataset.classes else 1.0)
        best_class = max(class_report.keys(), 
                        key=lambda x: class_report[x]['f1-score'] if x in train_dataset.classes else 0.0)
        
        if worst_class in train_dataset.classes and best_class in train_dataset.classes:
            print(f"🔍 Classe amb millor rendiment: {best_class} (F1: {class_report[best_class]['f1-score']:.3f})")
            print(f"⚠️ Classe a millorar: {worst_class} (F1: {class_report[worst_class]['f1-score']:.3f})")
        
        print("\n🚀 SEGÜENTS PASSOS:")
        print("1. Integrar model en sistema físic de la paperera")
        print("2. Provar amb imatges reals capturades per la càmera")
        print("3. Ajustar threshold de confiança per decisions")
        print("4. Implementar sistema de feedback per millorar model")
        
        # Estadístiques detallades per al desenvolupador
        print(f"\n📋 ESTADÍSTIQUES DETALLADES:")
        print("-" * 50)
        total_test_samples = len(test_metrics['true_labels'])
        correct_predictions = sum(1 for true, pred in zip(test_metrics['true_labels'], test_metrics['predictions']) if true == pred)
        print(f"📊 Total mostres test: {total_test_samples}")
        print(f"✅ Prediccions correctes: {correct_predictions}")
        print(f"❌ Prediccions incorrectes: {total_test_samples - correct_predictions}")
        print(f"🎯 Accuracy global: {test_metrics['accuracy']*100:.2f}%")
        
        # Mostrar matriu de confusió en text
        cm = confusion_matrix(test_metrics['true_labels'], test_metrics['predictions'])
        print(f"\n🔢 MATRIU DE CONFUSIÓ (format text):")
        print("     ", "  ".join([f"{cls[:8]:>8}" for cls in train_dataset.classes]))
        for i, class_name in enumerate(train_dataset.classes):
            row = " ".join([f"{cm[i][j]:8d}" for j in range(len(train_dataset.classes))])
            print(f"{class_name[:8]:>8} {row}")
    
    else:
        print("⚠️ No es va trobar el model guardat")
        
else:
    print("❌ No es van completar trials exitosos")
    print("🔧 Verifica:")
    print("   - Ruta del dataset: 'images_reorganitzades/'")
    print("   - Estructura: images_reorganitzades/[classe]/[imatges]")
    print("   - Memòria GPU disponible")
    print("   - Format d'imatges (PNG/JPG)")

# Informació addicional sobre el sistema
print("\n" + "="*70)
print("📋 INFORMACIÓ DEL SISTEMA DE PAPERERA INTEL·LIGENT")
print("="*70)
print("🗂️ Classes de residus:")
for i, class_name in enumerate(['envasos', 'paper', 'rebuig', 'vidre']):
    print(f"   {i}: {class_name}")

print("\n🔧 Configuració utilitzada:")
print("   📁 Estructura dataset: Plana reorganitzada")
print("   ⚖️ Sampler: WeightedRandomSampler per equilibrar")
print("   🔄 Augmentació: Només en entrenament")
print("   📊 Split: 60% train / 20% val / 20% test")
print("   🎯 Optimització: TPE + MedianPruner")

print(f"\n🏁 FIN DEL PROCÉS D'OPTIMITZACIÓ")
print("🤖 Model llest per integrar en la paperera intel·ligent!")

# Cleanup final
torch.cuda.empty_cache()
gc.collect()

🚀 INICIANDO OPTUNA HYPERPARAMETER TUNING - PAPERERA INTEL·LIGENT
📂 Carregant datasets amb estructura reorganitzada...
   🔄 Creant dataset train amb estructura plana...
   ✅ Dataset train creat: 9000 imatges
   📊 Distribució per classe: {'envasos': 3900, 'paper': 1500, 'rebuig': 2700, 'vidre': 900}
   🔄 Creant dataset val amb estructura plana...
   ✅ Dataset val creat: 3000 imatges
   📊 Distribució per classe: {'envasos': 1300, 'paper': 500, 'rebuig': 900, 'vidre': 300}
   🔄 Creant dataset test amb estructura plana...
   ✅ Dataset test creat: 3000 imatges
   📊 Distribució per classe: {'envasos': 1300, 'paper': 500, 'rebuig': 900, 'vidre': 300}
📊 Dataset: Train=9000, Val=3000, Test=3000 | 4 classes
🏷️ Classes: ['envasos', 'paper', 'rebuig', 'vidre']


[I 2025-09-01 12:18:38,896] A new study created in memory with name: waste_classification_optimization


📊 Gràfic de distribució guardat a: class_distribution.png
⚖️ Calculant pesos per equilibrar classes...
   📈 Pesos calculats per 9000 mostres d'entrenament
🎯 Objectiu: Optimitzar classificació de residus amb dataset equilibrat
💻 Device: CUDA
⚖️ Estratègia: WeightedRandomSampler per equilibrar classes
🔄 Augmentació: Dinàmica només en entrenament
⏱️ Temps estimat: 3-4 hores per 20 trials
🚀 Iniciant optimització...

🔍 Trial 1/20
   📊 Paràmetres: lr=0.00160, bs=48, epochs=5, dropout=0.7, weight_factor=0.7
   🎯 Baseline a superar: validation loss anterior
   🚀 Iniciant entrenament amb WeightedSampler...


   Epoch 1: Train=1.9584(26.2%) | Val=1.3923(10.1%) ⬆️
   🏆 ¡NOU RÈCORD GLOBAL! Val Loss: 1.392342


   Epoch 2: Train=1.3847(25.0%) | Val=1.3936(10.0%)


...

[I 2025-09-01 14:40:24,293] Trial 19 pruned.                                                                           


   Epoch 3: Train=1.3259(36.7%) | Val=1.3436(27.2%)
   ✂️ Trial podat - no és prometedor

🎉 OPTIMITZACIÓ COMPLETADA - PAPERERA INTEL·LIGENT
🏆 MILLOR RESULTAT:
   📊 Validation Loss: 0.880915
   ⚙️ Paràmetres: {'learning_rate': 0.0003454842518803514, 'batch_size': 24, 'num_epochs': 8, 'dropout': 0.3, 'weight_factor': 0.8}
   🔢 Trial número: 15

📈 Resum de trials:
   ✅ Trials completats: 20
   ✂️ Trials podats: 11
   ❌ Trials fallits: 0

🧪 AVALUACIÓ EN TEST SET:
--------------------------------------------------
🔍 Evaluant model en dataset de test...


Avaluació: 100%|███████████████████████████████████████████████████████████████████████| 94/94 [00:17<00:00,  5.28it/s]


📊 Test Loss: 0.855474
🎯 Test Accuracy: 64.23%

📋 ACCURACY PER CLASSE:
   envasos : P=0.751 R=0.628 F1=0.684
   paper   : P=0.613 R=0.720 F1=0.662
   rebuig  : P=0.618 R=0.629 F1=0.623
   vidre   : P=0.451 R=0.617 F1=0.521

💾 GENERANT INFORMES:
--------------------------------------------------
📊 Matriu de confusió guardada a: paperera_confusion_matrix.png
📄 Informe complet guardat a: paperera_optimization_report.json
📈 Gràfic d'optimització guardat a: paperera_optimization_history.png
✅ Arxius generats:
   📦 paperera_model_final_acc64.2.pth - Millor model optimitzat
   🔧 paperera_model_config.json - Configuració del model
   📊 paperera_confusion_matrix.png - Matriu de confusió
   📄 paperera_optimization_report.json - Informe complet
   📈 paperera_optimization_history.png - Història d'optimització
   📊 class_distribution.png - Distribució de classes

🎊 ¡PROCÉS COMPLETAT EXITOSAMENT!

💡 RECOMANACIONS PER LA PAPERERA INTEL·LIGENT:
--------------------------------------------------
✅ Model

101